In [ ]:
import pandas as pd
import glob
import os

folder_path = r"C:\Users\Robin\Whatsapp_chats"  #Enter your folderpath Folder path here

# Returns a list with all file_paths in the folder with .txt ending
all_files = glob.glob(os.path.join(folder_path, "*.txt"))

# Create empty list for DataFrames
dfs = []

# Read all filed, add the file name as column and append these to the list dfs
for file in all_files:
    df = pd.read_csv(file, sep='\t', engine='python', header=None, names=['Text'], on_bad_lines='warn', encoding='utf-8')
    df['Filename'] = os.path.basename(file)  # Dateiname hinzufügen
    dfs.append(df)

# Combine all DataFrames with the concat command.
chat = pd.concat(dfs, ignore_index=True)

# Define Pattern for Message rows
pattern = r'\d{2}\.\d{2}\.\d{2}, \d{2}:\d{2} - .+: .*'

# Filter the rows
chat = chat[chat['Text'].str.match(pattern)]
chat = chat[~chat['Text'].str.contains(r'(<Media Excluded>|null)')]

# Extract Date
chat['Date'] = chat['Text'].str.split(' - ').str[0]

# Extract Author
chat['Author']=  chat.apply(lambda row: row['Text'].replace(row['Date'] + ' - ', ''), axis=1)

# Extract Chat - Adjust "WhatsApp-Chat mit " with your filename pattern
chat['Chat'] = chat['Filename'].str.replace('WhatsApp-Chat mit ', '').str.replace('.txt', '')

# Split Author column to Author and Message
chat[['Author', 'Message']] = chat['Author'].str.split(': ', n=1, expand=True)

# Filter relevant columns
chat = chat[['Date','Author','Message','Chat']]